In [ ]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding_function = OpenAIEmbeddings()

docs = [
    Document(page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}),
    Document(page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}),
]


db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

In [ ]:
retriever.invoke("What exactly?")

In [ ]:
from langchain.prompts.prompt import PromptTemplate

rephrase_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
REPHRASE_TEMPLATE = PromptTemplate.from_template(rephrase_template)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

rephrase_chain = REPHRASE_TEMPLATE | ChatOpenAI(temperature=0) | StrOutputParser()

In [ ]:
rephrase_chain.invoke(
    {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | ANSWER_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

In [ ]:
final_chain = rephrase_chain | retrieval_chain

In [ ]:
final_chain.invoke(
    {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)

### Chat with returning documents

In [40]:
retrieved_documents =  {"docs": retriever, "question": RunnablePassthrough()}
final_inputs = {
    "context": lambda x: "\n".join(doc.page_content for doc in x["docs"]),
    "question": RunnablePassthrough(),
}
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI() | StrOutputParser(),
    "docs": RunnablePassthrough(),
}

final_chain = rephrase_chain | retrieved_documents | answer

In [41]:
result = final_chain.invoke(
        {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)
print(result)

{'answer': 'No, based on the given context, it is not true that the dog likes to eat tuna. The dog loves to eat pizza.', 'docs': {'docs': [Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}), Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}), Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'}), Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'})], 'question': 'Is it really true that the dog likes to eat tuna?'}}


In [42]:
result["answer"]

'No, based on the given context, it is not true that the dog likes to eat tuna. The dog loves to eat pizza.'

In [44]:
result["docs"]["docs"]

[Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}),
 Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}),
 Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'}),
 Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'})]